In [1]:
import numpy as np
import pandas as pd
import os, shutil, json
from PIL import Image
from pathlib import Path
from statistics import mean 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer

abs_path = r'%s' % os.getcwd().replace('\\','/')

In [2]:
tf.__version__

'2.2.0-rc2'

# Modelo Genérica

In [3]:
# Create the discriminator
discriminator = Sequential([
    InputLayer(input_shape=(64, 64, 3)),
    layers.Conv2D(128, (3, 3)),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128, (4, 4), strides=(2, 2)),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128, (4, 4), strides=(2, 2)),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128, (4, 4), strides=(2, 2)),
    layers.LeakyReLU(alpha=0.2),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
], name='discriminator')

# Create the generator
generator = Sequential([
    InputLayer(input_shape=(100,)),
    # We want to generate 128 coefficients to reshape into a 7x7x128 map
    layers.Dense(32 * 32 * 128),
    layers.LeakyReLU(alpha=0.2),
    layers.Reshape((32, 32, 128)),

    layers.Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same'),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2DTranspose(256, (5, 5), padding='same'),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2DTranspose(256, (5, 5), padding='same'),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(3, (7, 7), padding='same', activation='tanh')
], name='generator')

In [4]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        #self.NPZ_file_path = os.path.join(abs_path, "2_imagens_Mckinsey666_"+str(self.height)+"_"+str(self.height)+".npz").replace('\\','/')
        
    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        
    def get_images_as_nparray(self):
        with np.load(self.NPZ_file_path) as data:
            imagens_x = data['x']
        x_train = (imagens_x.reshape((imagens_x.shape[0],) + (self.height, self.width, self.channels)).astype('float32') - 127.5 ) / 127.5
        return x_train
        
    def train_step(self, data):
        # Unpack the data
        #data = self.get_images_as_nparray()
        real_images = data[0]
        
        # Sample random points in the latent space
        #batch_size = tf.shape(real_images)[0]
        batch_size = 32
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)
        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)
        # Assemple labels discriminating the real from fake images
        labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))
        
        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels,predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))
        
        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))
        
        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels,predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))
        return {'d_loss': d_loss, 'g_loss': g_loss}

In [5]:
NPZ_file_path = os.path.join(abs_path, "2_imagens_Mckinsey666_64_64.npz").replace('\\','/')
def get_images_as_nparray():
    with np.load(NPZ_file_path) as data:
        imagens_x = data['x']
    x_train = (imagens_x.reshape((imagens_x.shape[0],) + (64, 64, 3)).astype('float32') - 127.5 ) / 127.5
    return imagens_x
dataset = get_images_as_nparray()

In [6]:
latent_dim = 100

gan = GAN(discriminator, generator, latent_dim)
#gan.compile(d_optimizer=Adam(learning_rate=0.0003), g_optimizer=Adam(learning_rate=0.0003), optimizer='adam', loss_fn='binary_crossentropy')
gan.compile(d_optimizer=keras.optimizers.Adam(), 
            g_optimizer=keras.optimizers.Adam(), 
            loss_fn=keras.losses.BinaryCrossentropy(from_logits=True))
gan.fit(dataset, epochs=10000)

Epoch 1/10000
   4/1987 [..............................] - ETA: 7:54:36 - d_loss: 0.6647 - g_loss: 1.1237

KeyboardInterrupt: 